In [1]:
using Pkg
Pkg.activate("CT")
using CT
include("global_adder_passthrough.jl")

# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random

L = 12
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# initial_state = productMPS(qubit_site, [1,2,1,2,2,1,2,1,2,1,2,1,1,1,2,2])
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);

# initialize haining adder mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# initialize tao adder mpo
carry_links, T_vec, id_vec, gate_vec = initialize_links(L, qubit_site, shift_1_3_bits, ram_phy);
initial_state_1 = copy(initial_state);
initial_state_2 = copy(initial_state);

  Activating project at `~/CT_MPS_mini/CT`


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 12, 2, 11, 3, 10, 4, 9, 5, 8, 6, 7]
[1, 3, 5, 7, 9, 11, 12, 10, 8, 6, 4, 2]


In [2]:
# Performance Benchmark Configuration
using LinearAlgebra

# Print system information
println("Julia version: ", VERSION)
println("Available CPU cores: ", Sys.CPU_THREADS)
println("Julia threads: ", Threads.nthreads())

# Set BLAS to single-threaded for reproducible benchmarks
BLAS.set_num_threads(1)
println("BLAS threads: ", BLAS.get_num_threads())

# Optional: Set MKL threads if using MKL
try
    using MKL
    MKL.set_num_threads(1)
    println("MKL threads: ", MKL.get_num_threads())
catch
    println("MKL not available")
end

# Configure BenchmarkTools for consistency
using BenchmarkTools
BenchmarkTools.DEFAULT_PARAMETERS.samples = 50      # Reduced for faster iteration
BenchmarkTools.DEFAULT_PARAMETERS.evals = 1         # Single evaluation per sample
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 60      # Reduced time budget
BenchmarkTools.DEFAULT_PARAMETERS.gctrial = false   # Disable GC between trials
BenchmarkTools.DEFAULT_PARAMETERS.gcsample = false  # Disable GC between samples

println("Benchmark configuration complete!")


Julia version: 1.9.4
Available CPU cores: 8
Julia threads: 8
BLAS threads: 1
MKL threads: 1
Benchmark configuration complete!


In [3]:
# Prepare for benchmarking
using BenchmarkTools

# Force garbage collection before benchmarks for clean memory state
GC.gc()
GC.gc()  # Run twice to ensure cleanup

# Display current benchmark settings
println("Current BenchmarkTools settings:")
println("  samples: ", BenchmarkTools.DEFAULT_PARAMETERS.samples)
println("  evals: ", BenchmarkTools.DEFAULT_PARAMETERS.evals) 
println("  seconds: ", BenchmarkTools.DEFAULT_PARAMETERS.seconds)
println("  gctrial: ", BenchmarkTools.DEFAULT_PARAMETERS.gctrial)
println("  gcsample: ", BenchmarkTools.DEFAULT_PARAMETERS.gcsample)

println("\nSystem ready for benchmarking!")


Current BenchmarkTools settings:
  samples: 50
  evals: 1
  seconds: 60.0
  gctrial: false
  gcsample: false

System ready for benchmarking!


In [4]:
initial_state_1 = apply(add1_3,initial_state_1);
global_adder(initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)

MPS
[1] ((dim=2|id=978|"Qubit,Site,n=1"), (dim=3|id=369|"Link,l=1"))
[2] ((dim=2|id=835|"Qubit,Site,n=2"), (dim=3|id=369|"Link,l=1"), (dim=9|id=203|"Link,l=2"))
[3] ((dim=9|id=203|"Link,l=2"), (dim=2|id=554|"Qubit,Site,n=3"), (dim=15|id=672|"Link,l=3"))
[4] ((dim=15|id=672|"Link,l=3"), (dim=2|id=115|"Qubit,Site,n=4"), (dim=25|id=397|"Link,l=4"))
[5] ((dim=25|id=397|"Link,l=4"), (dim=2|id=333|"Qubit,Site,n=5"), (dim=45|id=495|"Link,l=5"))
[6] ((dim=45|id=495|"Link,l=5"), (dim=2|id=367|"Qubit,Site,n=6"), (dim=81|id=205|"Link,l=6"))
[7] ((dim=81|id=205|"Link,l=6"), (dim=2|id=759|"Qubit,Site,n=7"), (dim=96|id=104|"Link,l=7"))
[8] ((dim=96|id=104|"Link,l=7"), (dim=2|id=414|"Qubit,Site,n=8"), (dim=48|id=788|"Link,l=8"))
[9] ((dim=48|id=788|"Link,l=8"), (dim=2|id=659|"Qubit,Site,n=9"), (dim=24|id=662|"Link,l=9"))
[10] ((dim=24|id=662|"Link,l=9"), (dim=2|id=983|"Qubit,Site,n=10"), (dim=12|id=51|"Link,l=10"))
[11] ((dim=12|id=51|"Link,l=10"), (dim=2|id=189|"Qubit,Site,n=11"), (dim=2|id=700|"Lin

In [7]:
@benchmark initial_state_1 = apply(add1_3,$initial_state_1; cutoff=1e-10, maxdim=typemax(Int));


In [ ]:

@benchmark global_adder($initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy; cutoff=1e-10, maxdim=typemax(Int))


In [6]:
# Profile.print(format=:flat, sortedby=:count)

# # For now, just run the function without profiling:
# # global_adder(initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)